# Ensemble Challenge
Goal: to capture the complexity and nuances around the evolution of the pandemic at various stages and locations.

## Consider the following settings:
1. *Timepoint 1*: May 1st, 2020. Setting: Michigan State at the beginning of the pandemic when masking was the main preventative measure. No vaccines available.
2. *Timepoint 2*: May 1st, 2021. Setting: Michigan State prior to the arrival of the Delta variant. Vaccines available.
3. *Timepoint 3*: December 15th, 2021. Setting: Michigan State during the start of the first Omicron wave.

4. *BONUS*: Consider the same three time points, but change the setting to Louisiana, which had different COVID-19 dynamics compared to the Northern and Northeastern states.

## ...and related questions for each:
1. What is the most relevant data to use for model calibration?
2. What was our understanding of COVID-19 viral mechanisms at the time? For example, early in the pandemic, we didn't know if reinfection was a common occurance, or even possible.
3. What are the parameters related to contagiousness/transmissibility and severity of the dominant strain at the time?
4. What policies were in place for a stated location, and how can this information be incorporated into models? (See https://www.bsg.ox.ac.uk/research/covid-19-government-response-tracker for time series of interventions.)

## For each setting:
1. (a) Take a single model, calibrate it using historical data prior to the given date, and create a 4-week forecast for cases, hospitalizations, and deaths beginning on the given date. (b) Evaluate the forecast using the COVID-19 Forecasting Hub Error Metrics (WIS, MAE). The single model evaluation should be done in the same way as the ensemble.

2. Repeat (1), but with an ensemble of different models.

    a. It is fine to calibrate each model independently and weight naively.
    
    b. It would also be fine to calibrate the ensemble as a whole, assigning weights to the different component models, so that you minimize the error of the ensemble vs. historical data.
    
    c. Use the calibration scores and error metrics computed by the CDC Forecasting Hub. As stated on their [website](https://covid19forecasthub.org/doc/reports/): 
    
    “Periodically, we evaluate the accuracy and precision of the [ensemble forecast](https://covid19forecasthub.org/doc/ensemble/) and component models over recent and historical forecasting periods. Models forecasting incident hospitalizations at a national and state level are evaluated using [adjusted relative weighted interval scores (WIS, a measure of distributional accuracy)](https://arxiv.org/abs/2005.12881), and adjusted relative mean absolute error (MAE), and calibration scores. Scores are evaluated across weeks, locations, and targets. You can read [a paper explaining these procedures in more detail](https://www.medrxiv.org/content/10.1101/2021.02.03.21250974v1), and look at [the most recent monthly evaluation reports](https://covid19forecasthub.org/eval-reports). The final report that includes case and death forecast evaluations is 2023-03-13.” 

3. Produce the forecast outputs in the format specified by the CDC forecasting challenge, including the specified quantiles.

## Data
Use the following data sources:
1. Cases: [Johns Hopkins](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv), [Reich Lab](https://github.com/reichlab/covid19-forecast-hub/blob/master/data-truth/truth-Incident%20Cases.csv) (pulled from Johns Hopkins, but formatted)

2. Hospitalizations: [HealthData.gov](https://healthdata.gov/Hospital/COVID-19-Reported-Patient-Impact-and-Hospital-Capa/g62h-syeh)

3. Deaths: [Johns Hopkins](https://github.com/reichlab/covid19-forecast-hub/blob/master/data-truth/truth-Incident%20Deaths.csv), [Reich Lab](https://github.com/reichlab/covid19-forecast-hub/blob/master/data-truth/truth-Cumulative%20Deaths.csv)

In [1]:
# Load dependencies and functions from utils file
from pyciemss.utils.toronto_hackathon_utils.toronto_ensemble_challenge_utils import *

### Set the region of interest and infectious period, get the DataFrame containing case and hospital census data, and death data for that region, and plot said data if desired

In [2]:
# Set the region of interest and infectious period, get the DataFrame for that region
US_region = "MI" # 2-letter state abbreviation string (or "US")
regional_population = 10050000 # Michigan: 10,050,000 / Louisiana: 4,624,000
infectious_period = 7 # duration of infectious period (in days)
plot_data = False # plot the data when true

# Note: source datasets are quite large, so this will take a minute to run
data = get_case_hosp_death_data(US_region = US_region, infectious_period = infectious_period, make_csv=False)
data = data.reset_index()
# print(data)

# FYI: hosp data starts around 07/14/2020 and is NaN before, case and death data ends 03/04/2023 and is NaN after

if plot_data:
    # Plot case census data
    plt.subplot(1, 3, 1)
    plt.plot(data.index, data["case_census"], 'o')
    plt.title("Case Census")

    # Plot hosp census data
    plt.subplot(1, 3, 2)
    plt.plot(data.index, data["hosp_census"], 'o')
    plt.title("Hospital Census")

    # Plot cumulative deaths
    plt.subplot(1, 3, 3)
    plt.plot(data.index, data["cumulative_deaths"], 'o')
    plt.title("Cumulative Deaths")

/Users/altu809/Projects/pyciemss/src/pyciemss/utils/toronto_hackathon_utils/toronto_ensemble_challenge_utils.py:27: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_cases = pd.read_csv(url)
/Users/altu809/Projects/pyciemss/src/pyciemss/utils/toronto_hackathon_utils/toronto_ensemble_challenge_utils.py:28: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  raw_cases['date'] = pd.to_datetime(raw_cases.date, infer_datetime_format=True)
/Users/altu809/Projects/pyciemss/src/pyciemss/utils/toronto_hackathon_utils/toronto_ensemble_challenge_utils.py:34: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org

### Set relevant dates, test and train intervals

In [3]:
# Set train start date at the 55th day of data to use most/best historic data available
train_start_date = str(data["date"][55]) # this is 03/17/2020

# Given timepoints will act as test start dates
timepoint1 = "2020-05-01" 
timepoint2 = "2021-05-01"
timepoint3 = "2021-12-15"

# Set test end dates 4 weeks after timepoints
test_end_date1 = "2020-05-29"
test_end_date2 = "2021-05-29"
test_end_date3 = "2022-01-12"

### Set up train and test data

In [4]:
# Gather train and test data corresponding to Timepoint 1
train_data1, train_cases1, train_timepoints1, test_cases1, test_timepoints1, all_timepoints1 = \
get_train_test_data(data, train_start_date, timepoint1, test_end_date1)
# all_timepoints1 = get_all_timepoints_forreal(all_timepoints1)
data_file1 = US_region + "_train_data_1.csv"
train_data_to_csv(train_data1, data_file1)

# Set the start time (for all simulations)
start_time = train_timepoints1[0] - 1e-5

# Gather train and test data corresponding to Timepoint 2
train_data2, train_cases2, train_timepoints2, test_cases2, test_timepoints2, all_timepoints2 = \
get_train_test_data(data, train_start_date, timepoint2, test_end_date2)
# all_timepoints2 = get_all_timepoints_forreal(all_timepoints2)
data_file2 = US_region + "_train_data_2.csv"
train_data_to_csv(train_data2, data_file2)

# Gather train and test data corresponding to Timepoint 3
train_data3, train_cases3, train_timepoints3, test_cases3, test_timepoints3, all_timepoints3 = \
get_train_test_data(data, train_start_date, timepoint3, test_end_date3)
# all_timepoints3 = get_all_timepoints_forreal(all_timepoints3)
data_file3 = US_region + "_train_data_3.csv"
train_data_to_csv(train_data2, data_file3)

## Models:
1. You may consider any of the models you have seen in the started kit, or 6-month hackathon and evaluation scenarios.

2. You may search for new models in the literature, or use TA2 model extension/transformation capabilities to modify models already in Terarium.

### Load dependencies for ensembling

In [5]:
# Load ensembling dependencies
import os
from pyciemss.PetriNetODE.interfaces import load_petri_model, load_and_calibrate_and_sample_petri_model
from pyciemss.Ensemble.interfaces import load_and_sample_petri_ensemble, load_and_calibrate_and_sample_ensemble_model
from pyciemss.visuals import plots

### Get models to be ensembled

In [17]:
FIRST_PATH = "../Examples_for_TA2_Model_Representation/"

# # Model 1
# filename1 = "SEIARHDS_AMR.json"
# filename1 = os.path.join(FIRST_PATH, filename1)
# model1 = load_petri_model(filename1, add_uncertainty=True)

# # Model 2
# filename2 = "SEIARHD_AMR.json"
# filename2 = os.path.join(FIRST_PATH, filename2)
# # model2 = load_petri_model(filename2, add_uncertainty=True)

# # Model 3
# filename3 = "SIRHD_AMR.json"
# filename3 = os.path.join(FIRST_PATH, filename3)
# # model3 = load_petri_model(filename3, add_uncertainty=True)

# model_paths = [filename1, filename2, filename3]

##### Using hackathon models

# Model 1
filename1 = "scenario1_a.json"
model1 = load_petri_model(filename1, add_uncertainty=True)

# Model 2
filename2 = "scenario1_c.json"
model2 = load_petri_model(filename2, add_uncertainty=True)

# Model 3
filename3 = "scenario1_d.json"
model3 = load_petri_model(filename3, add_uncertainty=True)

model_paths = [filename1, filename2, filename3]

/Users/altu809/Projects/pyciemss/src/pyciemss/PetriNetODE/base.py:330: UserWarning: Parameter (('S', ('identity', 'ido:0000514')), ('E', ('identity', 'apollosv:0000154')), ('I', ('identity', 'ido:0000511')), 'ControlledConversion', 'rate') has value None and will be set to Uniform(0, 1)
  warnings.warn(warnings_string)
/Users/altu809/Projects/pyciemss/src/pyciemss/PetriNetODE/base.py:330: UserWarning: Parameter (('I', ('identity', 'ido:0000511')), ('R', ('identity', 'ido:0000592')), 'NaturalConversion', 'rate') has value None and will be set to Uniform(0, 1)
  warnings.warn(warnings_string)
/Users/altu809/Projects/pyciemss/src/pyciemss/PetriNetODE/base.py:330: UserWarning: Parameter (('I', ('identity', 'ido:0000511')), ('D', ('identity', 'ncit:C28554')), 'NaturalConversion', 'rate') has value None and will be set to Uniform(0, 1)
  warnings.warn(warnings_string)


In [ ]:
model1

### Create functions to define solution mapping dictionaries

In [7]:
# Define type of solution mapping required by each model
def solution_mapping1(model_solution: dict) -> dict:
    # solution mapping for model 1: SEIARHDS
    mapped_dict = {}
    mapped_dict["Cases"] = model_solution["symptomatic_population"] + model_solution["asymptomatic_population"]
    mapped_dict["Hospitalizations"] = model_solution["hospitalized_population"]
    mapped_dict["Deaths"] = model_solution["deceased_population"]
    return mapped_dict

def solution_mapping2(model_solution: dict) -> dict:
    # solution mapping for model 2: SEIARHD
    mapped_dict = {}
    mapped_dict["Cases"] = model_solution["symptomatic_population"] + model_solution["asymptomatic_population"]
    mapped_dict["Hospitalizations"] = model_solution["hospitalized_population"]
    mapped_dict["Deaths"] = model_solution["deceased_population"]
    return mapped_dict

def solution_mapping3(model_solution: dict) -> dict:
    # solution mapping for model 3: SIRHD
    mapped_dict = {}
    mapped_dict["Cases"] = model_solution["infectious_population"]
    mapped_dict["Hospitalizations"] = model_solution["hospitalized_population"]
    mapped_dict["Deaths"] = model_solution["deceased_population"]
    return mapped_dict

solution_mappings = [solution_mapping1, solution_mapping2, solution_mapping3]

# # FOR HACKATHON MODELS S E I R D
# mapped_data = {}
# mapped_data["timepoints"] = list(range(len(data.index)))
# mapped_data["I"] = data["case_census"]
# mapped_data["D"] = data["cumulative_deaths"]
# # write to CSV file
# mapped_data_df = pd.DataFrame(mapped_data)
# mapped_data_df.to_csv("mapped_data.csv", index=False)

### Create start states for each model at designated time points

In [8]:
# Define start states for each model
def create_start_state1(data, t_0, regional_population):
    '''Create the start state for Model 1 from data using our best guesses for
    mapping from observed variables to model state variables.'''
    
    start_state = data.set_index('date').loc[t_0].to_dict()
    returned_state = {}
    returned_state["exposed_population"] = start_state['case_census'] / 2
    if start_state['case_census'] <= 0:
        returned_state["symptomatic_population"] = 1
    else:
        returned_state["symptomatic_population"] = start_state['case_census'] / 4
    returned_state["asymptomatic_population"] = 3 * start_state['case_census'] / 4
    returned_state["recovered_population"] = 2 * start_state['case_census']
    
    if start_state["hosp_census"] > 0:
        returned_state["hospitalized_population"] = start_state["hosp_census"]
    else:
        returned_state["hospitalized_population"] = 0
    
    returned_state["deceased_population"] = start_state["cumulative_deaths"]
    returned_state["susceptible_population"] = regional_population - sum(returned_state.values())
    
    assert(returned_state["susceptible_population"] > 0)
    return returned_state #{k:v/regional_population for k, v in returned_state.items()}

def create_start_state2(data, t_0, regional_population):
    '''Create the start state for Model 2 from data using our best guesses for
    mapping from observed variables to model state variables.'''
    
    start_state = data.set_index('date').loc[t_0].to_dict()
    returned_state = {}
    returned_state["exposed_population"] = start_state['case_census'] / 2
    if start_state['case_census'] <= 0:
        returned_state["symptomatic_population"] = 1
    else:
        returned_state["symptomatic_population"] = start_state['case_census'] / 4
    returned_state["asymptomatic_population"] = 3 * start_state['case_census'] / 4
    returned_state["recovered_population"] = 2 * start_state['case_census']
    
    if start_state["hosp_census"] > 0:
        returned_state["hospitalized_population"] = start_state["hosp_census"]
    else:
        returned_state["hospitalized_population"] = 0
    
    returned_state["deceased_population"] = start_state["cumulative_deaths"]
    returned_state["susceptible_population"] = regional_population - sum(returned_state.values())
    
    assert(returned_state["susceptible_population"] > 0)
    return returned_state #{k:v/regional_population for k, v in returned_state.items()}

def create_start_state3(data, t_0, regional_population):
    '''Create the start state for Model 3 from data using our best guesses for
    mapping from observed variables to model state variables.'''
    
    start_state = data.set_index('date').loc[t_0].to_dict()
    returned_state = {}
    if start_state['case_census'] <= 0:
        returned_state["infectious_population"] = 1
    else:
        returned_state["infectious_population"] = start_state['case_census']
    returned_state["recovered_population"] = 2 * start_state['case_census']
    
    if start_state["hosp_census"] > 0:
        returned_state["hospitalized_population"] = start_state["hosp_census"]
    else:
        returned_state["hospitalized_population"] = 0
    
    returned_state["deceased_population"] = start_state["cumulative_deaths"]
    returned_state["susceptible_population"] = regional_population - sum(returned_state.values())
    
    assert(returned_state["susceptible_population"] > 0)
    return returned_state #{k:v/regional_population for k, v in returned_state.items()}

start_states = [create_start_state1(data, train_start_date, regional_population), 
               create_start_state2(data, train_start_date, regional_population), 
               create_start_state3(data, train_start_date, regional_population)]

### Set up and sample an ensemble of 1- 3 models 

In [ ]:
# Check that you can load and sample an ensemble of 1-3 models (you can!)
weights = [1/3, 1/3, 1/3]
num_samples = 10
timepoints = all_timepoints1 #[0.0, 1.0, 2.0, 3.0, 4.0]

# Run sampling
ensemble_samples = load_and_sample_petri_ensemble(
    model_paths, weights, solution_mappings, num_samples, timepoints
)

ensemble_samples

### Load, calibrate, and sample an ensemble of 1 model

In [10]:
# Check that you can load and sample and calibrate an ensemble of 1-3 models (hooray! you can!)
DATA_PATH = "../hackathon_prep/"
data_filename = data_file1
data_path = os.path.join(DATA_PATH, data_filename)
weights = [1]
num_samples = 10
timepoints = all_timepoints1
num_iterations = 150

calibrated_ensemble_of1 = load_and_calibrate_and_sample_ensemble_model(
    [model_paths[0]],
    data_path,
    weights,
    [solution_mapping1],
    num_samples,
    timepoints,
    start_states=start_states,
    total_population=regional_population,
    start_time=start_time,
    num_iterations=num_iterations,
    verbose=True,
)

iteration 0: loss = 1990.6884303987026
iteration 25: loss = 1966.342480391264
iteration 50: loss = 1941.4100823104382
iteration 75: loss = 1929.2438255250454
iteration 100: loss = 1927.6002357900143
iteration 125: loss = 1916.5757782161236


,timepoint_id,sample_id,model_0/beta_param,model_0/delta_param,model_0/total_population_param,"model_0/(('susceptible_population', ('identity', 'ido:0000514')), ('exposed_population', ('identity', 'ido:0000594')), (('asymptomatic_population', ('identity', 'ido:0000569')), ('symptomatic_population', ('identity', 'ido:0000573'))), 'GroupedControlledConversion', 'rate')_param",model_0/alpha_param,model_0/pS_param,"model_0/(('exposed_population', ('identity', 'ido:0000594')), ('symptomatic_population', ('identity', 'ido:0000573')), 'NaturalConversion', 'rate')_param","model_0/(('exposed_population', ('identity', 'ido:0000594')), ('asymptomatic_population', ('identity', 'ido:0000569')), 'NaturalConversion', 'rate')_param",...,"model_0/(('symptomatic_population', ('identity', 'ido:0000573')), ('deceased_population', ('identity', 'ncit:C168970')), 'NaturalConversion', 'rate')_param",model_0/dh_param,model_0/los_param,"model_0/(('hospitalized_population', ('identity', 'ncit:C25179')), ('recovered_population', ('identity', 'ido:0000592')), 'NaturalConversion', 'rate')_param","model_0/(('hospitalized_population', ('identity', 'ncit:C25179')), ('deceased_population', ('identity', 'ncit:C168970')), 'NaturalConversion', 'rate')_param",model_0/tau_param,model_0_weight,Cases_sol,Hospitalizations_sol,Deaths_sol
0,0,0,0.58397,1.440415,99459.796875,0.498082,4.234266,0.664481,0.996423,0.520368,...,0.788139,0.106309,7.625232,0.090931,0.435593,31.501881,1.0,90.000298,8.021289e-05,1.000177
1,1,0,0.58397,1.440415,99459.796875,0.498082,4.234266,0.664481,0.996423,0.520368,...,0.788139,0.106309,7.625232,0.090931,0.435593,31.501881,1.0,85.837120,6.297642e+00,20.650764
2,2,0,0.58397,1.440415,99459.796875,0.498082,4.234266,0.664481,0.996423,0.520368,...,0.788139,0.106309,7.625232,0.090931,0.435593,31.501881,1.0,66.709732,7.252704e+00,34.158127
3,3,0,0.58397,1.440415,99459.796875,0.498082,4.234266,0.664481,0.996423,0.520368,...,0.788139,0.106309,7.625232,0.090931,0.435593,31.501881,1.0,52.308205,5.728289e+00,41.307587
4,4,0,0.58397,1.440415,99459.796875,0.498082,4.234266,0.664481,0.996423,0.520368,...,0.788139,0.106309,7.625232,0.090931,0.435593,31.501881,1.0,42.189373,3.908596e+00,44.960903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,68,9,0.55792,1.447596,106126.171875,0.816589,4.274491,0.674484,0.845219,0.102136,...,0.907567,0.107366,6.444215,0.418702,0.768701,31.494232,1.0,0.000333,5.140519e-12,56.483250
726,69,9,0.55792,1.447596,106126.171875,0.816589,4.274491,0.674484,0.845219,0.102136,...,0.907567,0.107366,6.444215,0.418702,0.768701,31.494232,1.0,0.000277,4.289552e-12,56.483208
727,70,9,0.55792,1.447596,106126.171875,0.816589,4.274491,0.674484,0.845219,0.102136,...,0.907567,0.107366,6.444215,0.418702,0.768701,31.494232,1.0,0.000232,3.579454e-12,56.483246
728,71,9,0.55792,1.447596,106126.171875,0.816589,4.274491,0.674484,0.845219,0.102136,...,0.907567,0.107366,6.444215,0.418702,0.768701,31.494232,1.0,0.000193,2.986905e-12,56.483215


### Load, calibrate, and sample an ensemble of 3 models

In [13]:
# Check that you can load and sample and calibrate an ensemble of 1-3 models (hooray! you can!)
DATA_PATH = "../hackathon_prep/"
data_filename = data_file1
data_path = os.path.join(DATA_PATH, data_filename)
weights = [] # set equal weights initially
for i in range(0, len(model_paths)):
    weights.append(1/len(model_paths)) 
num_samples = 10
timepoints = all_timepoints1
num_iterations = 150

calibrated_ensemble_of3 = load_and_calibrate_and_sample_ensemble_model(
    model_paths,
    data_path,
    weights,
    solution_mappings,
    num_samples,
    timepoints,
    start_states=start_states,
    total_population=regional_population,
    start_time=start_time,
    num_iterations=num_iterations,
    verbose=True,
)

/Users/altu809/Projects/pyciemss/src/pyciemss/PetriNetODE/base.py:330: UserWarning: Parameter (('susceptible_population', ('identity', 'ido:0000514')), ('exposed_population', ('identity', 'ido:0000594')), (('asymptomatic_population', ('identity', 'ido:0000569')), ('symptomatic_population', ('identity', 'ido:0000573'))), 'GroupedControlledConversion', 'rate') has value None and will be set to Uniform(0, 1)
  warnings.warn(warnings_string)
/Users/altu809/Projects/pyciemss/src/pyciemss/PetriNetODE/base.py:330: UserWarning: Parameter (('exposed_population', ('identity', 'ido:0000594')), ('symptomatic_population', ('identity', 'ido:0000573')), 'NaturalConversion', 'rate') has value None and will be set to Uniform(0, 1)
  warnings.warn(warnings_string)
/Users/altu809/Projects/pyciemss/src/pyciemss/PetriNetODE/base.py:330: UserWarning: Parameter (('exposed_population', ('identity', 'ido:0000594')), ('asymptomatic_population', ('identity', 'ido:0000569')), 'NaturalConversion', 'rate') has valu

iteration 0: loss = 2081.9159725904465
iteration 25: loss = 2000.0357356071472
iteration 50: loss = 1293.9957889318466
iteration 75: loss = 1190.7821530103683
iteration 100: loss = 1201.054715514183
iteration 125: loss = 1199.004912495613


NameError: name 'result' is not defined